In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/Users/yunlei/Desktop/MGMT 478/Combined dataset_nonsort.csv')

In [3]:
data.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,PRCP,SNOW,TAVG,TMAX,TMIN
0,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",40.41236,-86.94739,181.7,2010-01,8.5,1.01,NaN,23.5,29.5,17.5
1,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",40.41236,-86.94739,181.7,2010-02,7.6,0.61,NaN,26,32.8,19.1
2,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",40.41236,-86.94739,181.7,2010-03,7.2,3.22,NaN,44.8,55.1,34.6
3,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",40.41236,-86.94739,181.7,2010-04,8.1,2.49,NaN,58.1,70.4,45.8
4,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",40.41236,-86.94739,181.7,2010-05,6.7,5.55,NaN,64.7,75.2,54.2


In [4]:
data['DATE'] = pd.to_datetime(data['DATE'])
data['YEAR'] = data['DATE'].dt.year
data['MONTH'] = data['DATE'].dt.month

In [5]:
station_unique = data['STATION'].unique().tolist()

In [6]:
data = data[['STATION','NAME','YEAR','MONTH','PRCP']]

In [7]:
data.isnull().sum()

STATION    0
NAME       0
YEAR       0
MONTH      0
PRCP       8
dtype: int64

In [8]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
data[['PRCP']] = imputer.fit_transform(data[['PRCP']])

In [9]:
data.isnull().sum()

STATION    0
NAME       0
YEAR       0
MONTH      0
PRCP       0
dtype: int64

In [10]:
weather_station_data = data[data['STATION']== 'USW00014835']

In [11]:
weather_station_data

,STATION,NAME,YEAR,MONTH,PRCP
0,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",2010,1,1.01
1,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",2010,2,0.61
2,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",2010,3,3.22
3,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",2010,4,2.49
4,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",2010,5,5.55
...,...,...,...,...,...
163,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",2023,8,3.54
164,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",2023,9,0.59
165,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",2023,10,3.21
166,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",2023,11,0.48


In [12]:
weather_station_data['PRCP_lag_1_year'] = weather_station_data['PRCP'].shift(12)

/var/folders/cr/ls79phkx4vb_rmnkc6h2ddqh0000gn/T/ipykernel_68278/1927566781.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_station_data['PRCP_lag_1_year'] = weather_station_data['PRCP'].shift(12)


In [13]:
df = weather_station_data[(weather_station_data['YEAR'] >= 2015) & (weather_station_data['YEAR'] <= 2023)]

In [14]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(df['PRCP'], df['PRCP_lag_1_year'])
print(mse)

5.04803888888889


In [15]:
def get_basic_model_data(weather_station):
    '''
    From 2015 to 2023, actual data from last year was used as the precipitation projection for next year, 
    and then compared to this year's actual data to derive the mse.
    '''
    weather_station_data = data[data['STATION']== weather_station]
    weather_station_data['PRCP_lag_1_year'] = weather_station_data['PRCP'].shift(12)
    df=weather_station_data[weather_station_data['YEAR']>=2015]
    mse = mean_squared_error(df['PRCP'], df['PRCP_lag_1_year'])
    return mse

In [16]:
get_basic_model_data('USW00014835')

/var/folders/cr/ls79phkx4vb_rmnkc6h2ddqh0000gn/T/ipykernel_68278/4065321984.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_station_data['PRCP_lag_1_year'] = weather_station_data['PRCP'].shift(12)


5.04803888888889

In [17]:
station_counts = data.groupby('STATION').size()

In [18]:
stations_with_enough_records = station_counts[station_counts >= 167].index

In [19]:
valid_stations = data[
    (data['YEAR'] == 2010) & 
    (data['STATION'].isin(stations_with_enough_records))
][['STATION', 'NAME']].drop_duplicates()

In [20]:
valid_stations

,STATION,NAME
0,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US"
168,USW00014848,"SOUTH BEND AIRPORT, IN US"
336,USC00120784,"BLOOMINGTON INDIANA UNIVERSITY, IN US"
503,USW00014827,"FORT WAYNE INTERNATIONAL AIRPORT, IN US"
671,USW00093819,"INDIANAPOLIS INTERNATIONAL AIRPORT, IN US"
839,USW00053866,"SHELBYVILLE MUNICIPAL AIRPORT, IN US"
1007,USW00093810,"CARBONDALE SOUTHERN ILLINOIS AIRPORT, IL US"
1176,USW00094892,"CHICAGO WEST CHICAGO DUPAGE AIRPORT, IL US"
1345,USW00094846,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US"
1514,USC00114355,"ILLINOIS CITY DAM 16, IL US"


In [21]:
valid_stations['overall_mse'] = valid_stations.apply(lambda row: get_basic_model_data(row['STATION']), axis=1)

In [22]:
valid_stations

,STATION,NAME,overall_mse
0,USW00014835,"LAFAYETTE PURDUE UNIVERSITY AIRPORT, IN US",5.048039
168,USW00014848,"SOUTH BEND AIRPORT, IN US",7.061131
336,USC00120784,"BLOOMINGTON INDIANA UNIVERSITY, IN US",9.840054
503,USW00014827,"FORT WAYNE INTERNATIONAL AIRPORT, IN US",4.807214
671,USW00093819,"INDIANAPOLIS INTERNATIONAL AIRPORT, IN US",7.476287
839,USW00053866,"SHELBYVILLE MUNICIPAL AIRPORT, IN US",5.770738
1007,USW00093810,"CARBONDALE SOUTHERN ILLINOIS AIRPORT, IL US",8.369791
1176,USW00094892,"CHICAGO WEST CHICAGO DUPAGE AIRPORT, IL US",5.383623
1345,USW00094846,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",6.477995
1514,USC00114355,"ILLINOIS CITY DAM 16, IL US",5.635398
